In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.contrib.tensorboard.plugins import projector

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.6.0'

In [3]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
from time import time
from keras.utils import np_utils#one hot
import os
from skimage import data
from skimage import transform
import skimage
import numpy as np

# Download the dataset


def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                  if os.path.isdir(os.path.join(data_directory,d))]
    labels=[]
    images=[]
    for d in directories:
        label_directory = os.path.join(data_directory,d)
        file_names = [os.path.join(label_directory,f)
                     for f in os.listdir(label_directory)
                     if f.endswith('.jpg')]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(str(d))
    return images, labels






Using TensorFlow backend.


In [4]:
ROOT_PATH = '/home/tang/targa/BigSize1/'
train_data_directory = os.path.join(ROOT_PATH, "train")
validation_data_directory = os.path.join(ROOT_PATH, "val")
test_data_directory = os.path.join(ROOT_PATH, "test")
train_images,train_labels = load_data(train_data_directory)
test_images,test_labels = load_data(test_data_directory)
validation_images,validation_labels = load_data(validation_data_directory)

In [5]:
#encoding trY teY
from numpy import array
#from numpy import argmax
#from keras.utils import to_categorical
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example

#train_labels = np_utils.to_categorical(train_labels,num_classes = None)
#test_labels = np_utils.to_categorical(test_labels,num_classes = None)
trL,teL,valL=array(train_labels),array(test_labels),array(validation_labels)
label_encoder = LabelEncoder()

In [6]:
integer_encoded_trL = label_encoder.fit_transform(trL)
integer_encoded_teL = label_encoder.fit_transform(teL)
integer_encoded_valL = label_encoder.fit_transform(valL)

In [7]:
trainLabels = np_utils.to_categorical(integer_encoded_trL,num_classes = 32)
testLabels = np_utils.to_categorical(integer_encoded_teL,num_classes = 32)
validationLabels = np_utils.to_categorical(integer_encoded_valL,num_classes = 32)

In [8]:
num_sample = 1165
for n in range(33):
    nnnn = num_sample*n
    print(n)
    print(integer_encoded_teL[nnnn])
    print(teL[nnnn])
    print('--------------------')

0
18
J
--------------------
1
18
J
--------------------
2
14
E
--------------------
3
7
7
--------------------
4
10
A
--------------------
5
25
S
--------------------
6
11
B
--------------------
7
20
L
--------------------
8
26
T
--------------------
9
31
Z
--------------------
10
23
P
--------------------
11
19
K
--------------------
12
12
C
--------------------
13
15
F
--------------------
14
22
N
--------------------
15
28
W
--------------------
16
9
9
--------------------
17
30
Y
--------------------
18
1
1
--------------------
19
3
3
--------------------
20
4
4
--------------------
21
24
R
--------------------
22
5
5
--------------------
23
16
G
--------------------
24
27
V
--------------------
25
6
6
--------------------
26
0
0
--------------------
27
13
D
--------------------
28
21
M
--------------------
29
29
X
--------------------
30
8
8
--------------------
31
2
2
--------------------
32
17
H
--------------------


In [9]:
teL[1000]

'J'

In [10]:
testLabels[2000]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
######图片均衡化

In [ ]:
trainImages[0]

In [11]:
########convert rgb images into black and white
#convert images into size of vector
from skimage.color import rgb2gray

trainImages = rgb2gray(np.array(train_images))
testImages = rgb2gray(np.array(test_images))
valImages = rgb2gray(np.array(validation_images))
train_flat_images = np.reshape(trainImages,(len(trainImages),160*90))
test_flat_images = np.reshape(testImages,(len(testImages),160*90))
val_flat_images = np.reshape(valImages,(len(valImages),160*90))

In [12]:
valImages.shape

(37312, 160, 90)

In [13]:
train_flat_images[0].shape

(14400,)

In [14]:
print('train images shape:\t\t{}'.format(train_flat_images.shape))

print('test images shape:\t\t{}'.format(test_flat_images.shape))

print('val images shape:\t\t{}'.format(val_flat_images.shape))

train images shape:		(37312, 14400)
test images shape:		(37312, 14400)
val images shape:		(37312, 14400)


In [15]:
integer_encoded_trL[0]

18

In [16]:
validationLabels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
print('Size of:')
print('- Training-set:\t\t{}'.format(len(trainLabels)))
print('- Test-set:\t\t{}'.format(len(testLabels)))
print('- Validation-set:\t{}'.format(len(validation_labels)))

Size of:
- Training-set:		37312
- Test-set:		37312
- Validation-set:	37312


In [18]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [19]:
def create_sprite_image(images):
    """Returns a sprite image consisting of images passed as argument. Images should be count x width x height"""
    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    
    
    spriteimage = np.ones((img_h * n_plots ,img_w * n_plots ))
    
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                spriteimage[i * img_h:(i + 1) * img_h,
                  j * img_w:(j + 1) * img_w] = this_img
    
    return spriteimage

def vector_to_matrix_mnist(mnist_digits):
    """Reshapes normal mnist digit (batch,28*28) to matrix (batch,28,28)"""
    return np.reshape(mnist_digits,(-1,160,90))

def invert_grayscale(mnist_digits):
    """ Makes black white, and white black """
    return 1-mnist_digits

In [20]:
#batch_size = 100
max_steps = 1001
image_num = 500
# n_batch = len(trainLabels)//batch_size
# lr = tf.Variable(0.001,dtype = tf.float32)
LOG_DIR = '/home/DL/tensorflow/targaLogs/twoLayerVis/'
# NAME_TO_VISUALISE_VARIABLE = "targaembedding"
# TO_EMBED_COUNT = 500

In [21]:
testLabels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
test_flat_images.shape

(37312, 14400)

In [23]:
sess = tf.Session()
embedding = tf.Variable(tf.stack(test_flat_images[:image_num]),trainable=False,name='embedding')

def variable_summaries(var):
    with tf.name_scope('summeries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram',var)#直方图
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,14400],name='x-input')
    y = tf.placeholder(tf.float32,[None,32],name='y-input')
#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,160,90,1])#-1代表不确定，２８＊２８
    tf.summary.image('input',image_shaped_input,10)#放十张图片

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([14400,32]),name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([32]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels =y,logits = prediction))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

sess.run(tf.global_variables_initializer())


with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

#产生metadata
#如果存在元文件则删除
if tf.gfile.Exists(LOG_DIR + 'metadata.tsv'):
    tf.gfile.DeleteRecursively(LOG_DIR + 'metadata.tsv')
with open(LOG_DIR + 'metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(testLabels[:],1))
    for i in range(image_num):
        f.write(str(labels[i])+'\n')



merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(LOG_DIR,sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()

embed.tensor_name = embedding.name

embed.metadata_path = LOG_DIR + 'metadata.tsv'

######embedding copy
#just run one time
"""
batch_x,batch_y = next_batch(TO_EMBED_COUNT,train_flat_images,trainLabels)

path_for_targa_sprites =  os.path.join(LOG_DIR,'targa_32k_sprite.png')

to_visualise = batch_x
to_visualise = vector_to_matrix_mnist(to_visualise)
#to_visualise = invert_grayscale(to_visualise)

sprite_image = create_sprite_image(to_visualise)

#plt.imsave(path_for_targa_sprites,sprite_image,cmap='gray')
#plt.imshow(sprite_image,cmap='gray')
plt.imsave(path_for_targa_sprites,sprite_image)
plt.imshow(sprite_image)
"""







###########

embed.sprite.image_path = LOG_DIR + 'targa_32k_sprite.png'

embed.sprite.single_image_dim.extend([160,90])

projector.visualize_embeddings(projector_writer,config)

for i in range(max_steps):
    batch_xs,batch_ys = next_batch(100,train_flat_images,trainLabels)
    
    run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_ =sess.run([merged,train_step],feed_dict ={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d'%i)
    projector_writer.add_summary(summary,i)
    if i%100==0:
        acc = sess.run(accuracy,feed_dict={x:test_flat_images,y:testLabels})
        print('Iter'+str(i) + ',testing accuracy='+str(acc))
saver.save(sess,LOG_DIR+'a_model.ckpt',global_step=max_steps)
projector_writer.close()
sess.close()

batch_ys.shape

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter0,testing accuracy=0.031196399
Iter100,testing accuracy=0.031303603
Iter200,testing accuracy=0.04612457
Iter300,testing accuracy=0.08013508
Iter400,testing accuracy=0.1170937
Iter500,testing accuracy=0.12470519
Iter600,testing accuracy=0.13907054
Iter700,testing accuracy=0.14435035
Iter800,testing accuracy=0.14732526
Iter900,testing accuracy=0.17530553
Iter1000,testing accuracy=0.16965051


(100, 32)